# Лабораторная работа №5

## Упражнение 1

### Задание
Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [20]:
%%time

from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

x = inc(1)
y = inc(2)
z = add(x, y)

CPU times: user 1.94 ms, sys: 201 µs, total: 2.14 ms
Wall time: 3 s


### Решение

In [21]:
%%time

from dask import delayed

@delayed
def d_inc(x):
    sleep(1)
    return x + 1

@delayed
def d_add(x, y):
    sleep(1)
    return x + y

xd = d_inc(1)
yd = d_inc(2)
zd = d_add(xd, yd)

zd.compute()

CPU times: user 5.07 ms, sys: 251 µs, total: 5.32 ms
Wall time: 2.01 s


5

### Объяснение

Код, приведенный в задании, выполняется за ~3с, так как содержит 3 последовательных вызова функций, каждая из которых имеет задержку в 1с.

Код, приведенный в решении, оптимизирован таким образом, что все независимые вызовы функций (в нашем случае это `inc(1)` и `inc(2)`) распараллеливаются, что позволяет сократить время работы до 2с.

## Упражнение 2

### Задание

Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [22]:
%%time

def inc(x):
    sleep(1)
    return x + 1

def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results = []
for x in data:
    if is_even(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)

total = sum(results)
print(total)

90
CPU times: user 3.33 ms, sys: 345 µs, total: 3.67 ms
Wall time: 10 s


### Решение

In [37]:
%%time

from dask import delayed

def is_even(x):
    return not x % 2

@delayed
def d_inc(x):
    sleep(1)
    return x + 1

@delayed
def d_double(x):
    sleep(1)
    return 2 * x

results = [d_double(x) if is_even(x) else d_inc(x) for x in range(1, 11)]

total = delayed(sum)(results)

total.compute()

CPU times: user 2.01 ms, sys: 7.73 ms, total: 9.74 ms
Wall time: 2.01 s


90

### Объяснение

Код, приведенный в задании, выполняется за ~10с, так как в нем выполняется последовательная обработка массива из 10 элементов, каждый из которых обрабатывается в течение 1с. 

Код, приведенный в решении, на данной машине (имеется в виду *локальная* машина - не Google Colab) позволяет сократить время выполнения до 2с. Так как все операции обработки независимы друг от друга, они могут быть выполнены параллельно, а так как на данной машине присутствует 8 логических ядер, Dask может выполнять 8 одновременных операций, что в случае массива размером 10 означает 2 итерации (8 + 2).

## Упражнение 3

### Задание

Использовать dask.bag. 

Мы будем использовать пакет Dask для расчета частот последовательностей по пять символов, а затем сортировать последовательности в порядке убывания частоты встреч.

Реализуйте функцию group_characters(line, n=5) для группировки строки по n символов и возврата итератора. Строка — это текстовая строка в файле genome.txt.  (сгенерировать самостоятельно)

In [24]:
# >>> line = "abcdefghijklmno"
# >>> for seq in group_character(line, 5):
#         print(seq)
        
# "abcde"
# "efghi"
# "klmno"

Реализовать group_and_split(line)

In [25]:
# >>> group_and_split('abcdefghijklmno')
# ['abcde', 'fghij', 'klmno']

Использовать dask.bag для вычисления частот встреч групп символов по 5 и отсортировать по ней.

### Решение

In [26]:
# Генерация случайной строки

from random import choice
from string import ascii_lowercase

def generate_string(n):
    return ''.join(choice(ascii_lowercase[:4]) for _ in range(n))

with open("temp/genome.txt", "w") as f:
    f.write(generate_string(1000))

!cat ./temp/genome.txt

adcadaadcdccbcaccbaadbbcccaacbbcbccaaaabdbcaadcdbccbdbcadababbaaabdaddcbbcacdccdbaaccdabcdadaacdccdbbcdbccbcdacdccaccbddbabaacbbbacdbbacacdbbadbbacdabdadbaaabdacbbdddccaaaaddbcaaccccbbdbccacbcdacdbbadddddacabaabbdadaacccccdacbabacbadaaddaaacbdcbbaacbcccdcbbbdbabdaaacdccadacdcbcdbadbbdabdbddcccccccaddcdbdabcccbdcdbdccadbacaccbdadbcddabcbbdbbbabcbbbbdbdcdbcbdcdbbccdddcbbdaccaddbbbbacdbdaaccabdabbaabbaabaddaadbdaabdcbbdbaadcddcaccabdaccdcdcabdbaaacbcbdddddcadcaaacaaaadadbdbbaaaaddcddddbcaaabcccaddbdbdccaadaccbcacccbbaccababbcaabcadcbdbbdacbbadacdbcdcbcaaccbcaacbdaccbdabddddcccaadaaccacaaaadcbccaaadcdccabbbcabcbcabbadcddbdcbddadbabcdabdcaacdccdcabcbbbcabadbaaaccabaabaabcacbcadbbccdccdcabddcabbbcdbbbcbddccbcbbbcdabddcbaabacddbabcdbacddadddadcccaadaaaabdbaaadddbccdaaaacabbbddbdddbbcbabccccbdabccaccdbdcadadabcaccadddcaacbabaaadcdbabdaaaccadbbdadbdcacbbdcdbbbdadbaacbccbaccaacdadaabddabacdbbccdcbcddcacdbbcdaababadcbcbcdcbbdbbcdcabdccdaccbbbcdcbaaaddaaadbbaacdbddabdaaaddaaaddddaacdbbacabdbcdbbaa

In [27]:
def group_character(line, n=5):
    return (line[i : i + n] for i in range(0, len(line), n))


def group_and_split(line, n=5):
    return [line[i : i + n] for i in range(0, len(line), n)]


line = "abcdefghijklmno"
for seq in group_character(line, 5):
    print(seq)

group_and_split("abcdefghijklmno")


abcde
fghij
klmno


['abcde', 'fghij', 'klmno']

In [28]:
from dask import bag as db

with open("./temp/genome.txt") as f:
    line = f.read()

b = db.from_sequence(group_character(line))
f = b.frequencies(sort=True)

print(f.compute())

[('dbbcc', 3), ('caacb', 2), ('babaa', 2), ('cabaa', 2), ('bbdad', 2), ('abdaa', 2), ('cabda', 2), ('cdabd', 2), ('bcbbb', 2), ('bacdd', 2), ('aaadd', 2), ('adcad', 1), ('aadcd', 1), ('ccbca', 1), ('ccbaa', 1), ('bcbcc', 1), ('aaaab', 1), ('dbcaa', 1), ('dcdbc', 1), ('cbdbc', 1), ('adaba', 1), ('bbaaa', 1), ('bdadd', 1), ('cbbca', 1), ('cdccd', 1), ('baacc', 1), ('dabcd', 1), ('adaac', 1), ('dccdb', 1), ('bcdbc', 1), ('cbcda', 1), ('cdcca', 1), ('ccbdd', 1), ('cbbba', 1), ('cdbba', 1), ('cacdb', 1), ('badbb', 1), ('acdab', 1), ('dadba', 1), ('aabda', 1), ('cbbdd', 1), ('dccaa', 1), ('aaddb', 1), ('caacc', 1), ('ccbbd', 1), ('bccac', 1), ('bcdac', 1), ('dbbad', 1), ('dddda', 1), ('aaccc', 1), ('ccdac', 1), ('babac', 1), ('badaa', 1), ('ddaaa', 1), ('cbdcb', 1), ('baacb', 1), ('cccdc', 1), ('bbbdb', 1), ('acdcc', 1), ('adacd', 1), ('cbcdb', 1), ('adbbd', 1), ('abdbd', 1), ('dcccc', 1), ('cccad', 1), ('dcdbd', 1), ('abccc', 1), ('bdcdb', 1), ('dccad', 1), ('bacac', 1), ('cbdad', 1), ('bcd

# Лабораторная работа №6

### Упражнение 4

### Задание

Использовать dask.array

Вычислите сумму, произведение и среднее значение двумерного квадратного массива (сгенерировать случайно) из 1_000_000 элементов в каждом измерении.

1) с помощью dask (определить самостоятельно разбиение на блоки)

2) без dask

Измерить время

### Решение

In [29]:
%%timeit

from dask import array as da

dim = 10_000
chunk_size = 2000

xd = da.random.random((dim, dim), chunks=(chunk_size, chunk_size))

sd = xd.sum()
pd = xd.prod()

s = sd.compute()
p = pd.compute()
m = s / (dim * dim)

857 ms ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit

import numpy as np

dim = 10_000

x = np.random.random((dim, dim))

s = x.sum()
p = x.prod()
m = s / (dim * dim)

1.63 s ± 61.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Упражнение 5

### Задание

Постройте массив 20000x20000 нормально распределенных случайных значений (random), разбитых на фрагменты размером 1000x1000


Возьмите среднее по одной оси


Возьмите каждый 100-й элемент

Сделайте задание в двух вариантах:

1) dask

2) numpy

Измерите время и объясните результат.

### Решение

In [31]:
%%time

from dask import array as da

dim = 20_000
ch_s = 1_000

xd = da.random.normal(size=(dim, dim), chunks=(ch_s, ch_s))
md = da.mean(xd[::100], axis=0)

m = md.compute()

CPU times: user 20 s, sys: 227 ms, total: 20.2 s
Wall time: 2.97 s


In [32]:
%%time

import numpy as np

dim = 20_000

x = np.random.normal(size=(dim, dim))
m = np.mean(x[::100], axis=0)

CPU times: user 12.8 s, sys: 7.36 s, total: 20.2 s
Wall time: 20.9 s


### Объяснение

Dask работает быстрее, чем Numpy, так как он распараллеливает вычисления на каждом блоке, тогда как Numpy производит вычисления в одном процессе на всем массиве сразу.